In [1]:
%reload_ext autoreload
%autoreload 2
import os
import sys
from dotenv import load_dotenv, find_dotenv
from dataclasses import dataclass
from pathlib import Path
load_dotenv(find_dotenv())
sys.path.append(os.getenv("PROJECT_FOLDER"))
from src.utils.common import logger, read_yaml, create_directories

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    model_params: dict
    target_column: str

In [2]:
# os.chdir("..")

In [3]:
from src.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath: str = os.getenv("CONFIG_FILE_PATH"),
        params_filepath: str = os.getenv("PARAMS_FILE_PATH"),
        schema_filepath: str = os.getenv("SCHEMA_FILE_PATH"),
    ):
        self.config = read_yaml(Path(config_filepath))
        self.params = read_yaml(Path(params_filepath))
        self.schema = read_yaml(Path(schema_filepath))
        
        create_directories([self.config.artifacts_root])
    
    def get_model_trainer_config(self):
        """
        Get configuration for model training
        """
        config = self.config.model_trainer
        params = self.params.MultinomialNB
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])
        
        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            model_params = params,
            target_column = schema.name
        )
        
        return model_trainer_config

In [4]:
import joblib
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB

class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        """
        Instantiate `ModelTrainer` class

        Args:
            config (ModelTrainerConfig): Configuration for model training
        """
        self.config = config
    
    def train(self):
        """
        Train and save model
        """
        logger.info("Train model")
        train = pd.read_csv(self.config.train_data_path) 
        test = pd.read_csv(self.config.test_data_path)

        X_train = train.drop(columns=[self.config.target_column], axis=1).to_numpy().reshape(-1)
        y_train = train[self.config.target_column].to_numpy().reshape(-1)
        model = Pipeline([
            ('CountVectorizer', CountVectorizer()),
            ('MultinomialNB', MultinomialNB(**self.config.model_params))
        ])
        model.fit(X_train, y_train)
        logger.info("Save model")
        joblib.dump(model, self.config.root_dir+"/"+self.config.model_name)

In [5]:
try:
    configuration_manager = ConfigurationManager()
    model_trainer = ModelTrainer(config=configuration_manager.get_model_trainer_config())
    model_trainer.train()
except Exception as e:
    logger.error(e)

2024-03-08 13:49:01,832 - sentiment-classifier-logger - INFO - yaml file: D:\Documents\GitHub\customer-product-reviews-sentiment-classifier\config\config.yaml loaded successfully
2024-03-08 13:49:01,832 - sentiment-classifier-logger - INFO - yaml file: D:\Documents\GitHub\customer-product-reviews-sentiment-classifier\params.yaml loaded successfully
2024-03-08 13:49:01,840 - sentiment-classifier-logger - INFO - yaml file: D:\Documents\GitHub\customer-product-reviews-sentiment-classifier\schema.yaml loaded successfully
2024-03-08 13:49:01,842 - sentiment-classifier-logger - INFO - Created directory at: artifacts
2024-03-08 13:49:01,842 - sentiment-classifier-logger - INFO - Created directory at: artifacts/model_trainer
2024-03-08 13:49:01,842 - sentiment-classifier-logger - INFO - Train model
2024-03-08 13:49:02,433 - sentiment-classifier-logger - INFO - Save model
